<a href="https://colab.research.google.com/github/yvonwin/jupyter-notebooks/blob/main/%E8%AE%A1%E7%AE%97%E6%95%B0%E6%8D%AE%E9%9B%86%E7%9A%84%E5%9D%87%E5%80%BC%E5%92%8C%E6%A0%87%E5%87%86%E5%B7%AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch.serialization import load
import torchvision.datasets as datasets
import torchvision.transforms as tansformes
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

In [2]:
train_dataset = datasets.CIFAR10(root='CIFAR/', train=True, transform=transforms.ToTensor(),download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size = 64, shuffle=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting CIFAR/cifar-10-python.tar.gz to CIFAR/


In [3]:
def get_mean_std(loader):
    # Var[x] = E[X**2]-E[X]**2
    channels_sum,channels_squared_sum,num_batches = 0,0,0
    for data, _ in loader:
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1

    print(num_batches)
    print(channels_sum)
    mean = channels_sum/num_batches
    std = (channels_squared_sum/num_batches - mean**2) **0.5

    return mean,std


In [4]:
mean,std = get_mean_std(train_loader)

print(mean)
print(std)

782
tensor([384.2605, 377.0288, 349.1746])
tensor([0.4914, 0.4821, 0.4465])
tensor([0.2470, 0.2435, 0.2616])


直接从文件夹获取

In [4]:
!pip install -U albumentations
!pip install -U opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60.9 MB 108 kB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


In [8]:
# 进入谷歌硬盘空间 保存时保存到这个路径
#connect to self drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
unzip:  cannot find or open ./drive/MyDrive/HuBMAP_2022_256x256.zip, ./drive/MyDrive/HuBMAP_2022_256x256.zip.zip or ./drive/MyDrive/HuBMAP_2022_256x256.zip.ZIP.


In [11]:
!unzip drive/MyDrive/HuBMAP_2022\ 256x256.zip -d ./input/

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: ./input/masks/164_0005.png  
  inflating: ./input/masks/164_0006.png  
  inflating: ./input/masks/164_0007.png  
  inflating: ./input/masks/164_0008.png  
  inflating: ./input/masks/16564_0001.png  
  inflating: ./input/masks/16564_0002.png  
  inflating: ./input/masks/16564_0003.png  
  inflating: ./input/masks/16564_0004.png  
  inflating: ./input/masks/16564_0005.png  
  inflating: ./input/masks/16564_0007.png  
  inflating: ./input/masks/16564_0008.png  
  inflating: ./input/masks/16609_0000.png  
  inflating: ./input/masks/16609_0001.png  
  inflating: ./input/masks/16609_0002.png  
  inflating: ./input/masks/16609_0003.png  
  inflating: ./input/masks/16609_0004.png  
  inflating: ./input/masks/16609_0005.png  
  inflating: ./input/masks/16609_0006.png  
  inflating: ./input/masks/16609_0007.png  
  inflating: ./input/masks/16609_0008.png  
  inflating: ./input/masks/16659_0000.png  
  inflating: ./input/masks/16659_0001.png  
  inflating: 

In [12]:
import numpy as np

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

from pathlib import Path
import cv2

from tqdm import tqdm

####### PARAMS

device = torch.device('cpu')
num_workers = 4
image_size = 1024
batch_size = 8
data_path = Path('./input/train')
images_ext = ['*.png']

augs = A.Compose([A.Resize(height=image_size, width=image_size),
                  A.Normalize(mean=(0, 0, 0), std=(1, 1, 1)),
                  ToTensorV2()])


def get_all_files_in_folder(folder, types):
    files_grabbed = []
    for t in types:
        files_grabbed.extend(folder.rglob(t))
    files_grabbed = sorted(files_grabbed, key=lambda x: x)
    return files_grabbed


images = get_all_files_in_folder(data_path, images_ext)


class LeafData(Dataset):

    def __init__(self,
                 files,
                 transform=None):
        self.files = files
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        # import
        # path = os.path.join(self.directory, self.data.iloc[idx]['image_id'])
        image = cv2.imread(str(self.files[idx]), cv2.COLOR_BGR2RGB)

        # augmentations
        if self.transform is not None:
            image = self.transform(image=image)['image']

        return image


# dataset
image_dataset = LeafData(files=images,
                         transform=augs)

# data loader
image_loader = DataLoader(image_dataset,
                          batch_size=batch_size,
                          shuffle=False,
                          num_workers=num_workers,
                          pin_memory=True)

####### COMPUTE MEAN / STD

# placeholders
psum = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs in tqdm(image_loader):
    psum += inputs.sum(axis=[0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis=[0, 2, 3])

####### FINAL CALCULATIONS

# pixel count
count = len(images) * image_size * image_size

# mean and std
total_mean = psum / count
total_var = (psum_sq / count) - (total_mean ** 2)
total_std = torch.sqrt(total_var)

total_mean_255 = total_mean * 255
total_std_255 = total_std * 255

# output
print('mean: ' + str(total_mean))
print('std:  ' + str(total_std))
print('mean_255: ' + str(total_mean_255))
print('std_255:  ' + str(total_std_255))


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 349/349 [02:01<00:00,  2.88it/s]

mean: tensor([0.7636, 0.7455, 0.7717])
std:  tensor([0.2535, 0.2571, 0.2428])
mean_255: tensor([194.7261, 190.1119, 196.7947])
std_255:  tensor([64.6371, 65.5724, 61.9054])


测试512

In [13]:
# get data
! mkdir /root/.kaggle
! mv kaggle.json /root/.kaggle/
! chmod 600  /root/.kaggle/kaggle.json

!kaggle datasets download -d thedevastator/hubmap-2022-512x512
!mkdir ./input2
!unzip hubmap-2022-512x512.zip -d ./input2/

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: ./input2/masks/1878_0007.png  
  inflating: ./input2/masks/1878_0008.png  
  inflating: ./input2/masks/18792_0000.png  
  inflating: ./input2/masks/18792_0001.png  
  inflating: ./input2/masks/18792_0002.png  
  inflating: ./input2/masks/18792_0003.png  
  inflating: ./input2/masks/18792_0004.png  
  inflating: ./input2/masks/18792_0005.png  
  inflating: ./input2/masks/18792_0006.png  
  inflating: ./input2/masks/18792_0007.png  
  inflating: ./input2/masks/18792_0008.png  
  inflating: ./input2/masks/18900_0000.png  
  inflating: ./input2/masks/18900_0001.png  
  inflating: ./input2/masks/18900_0002.png  
  inflating: ./input2/masks/18900_0003.png  
  inflating: ./input2/masks/18900_0004.png  
  inflating: ./input2/masks/18900_0005.png  
  inflating: ./input2/masks/18900_0006.png  
  inflating: ./input2/masks/18900_0007.png  
  inflating: ./input2/masks/19048_0000.png  
  inflating: ./input2/masks/19048_0001.png  
  inflating: ./input2/masks/19

In [14]:
import numpy as np

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

from pathlib import Path
import cv2

from tqdm import tqdm

####### PARAMS

device = torch.device('cpu')
num_workers = 4
image_size = 1024
batch_size = 8
data_path = Path('./input2/train')
images_ext = ['*.png']

augs = A.Compose([A.Resize(height=image_size, width=image_size),
                  A.Normalize(mean=(0, 0, 0), std=(1, 1, 1)),
                  ToTensorV2()])


def get_all_files_in_folder(folder, types):
    files_grabbed = []
    for t in types:
        files_grabbed.extend(folder.rglob(t))
    files_grabbed = sorted(files_grabbed, key=lambda x: x)
    return files_grabbed


images = get_all_files_in_folder(data_path, images_ext)


class LeafData(Dataset):

    def __init__(self,
                 files,
                 transform=None):
        self.files = files
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        # import
        # path = os.path.join(self.directory, self.data.iloc[idx]['image_id'])
        image = cv2.imread(str(self.files[idx]), cv2.COLOR_BGR2RGB)

        # augmentations
        if self.transform is not None:
            image = self.transform(image=image)['image']

        return image


# dataset
image_dataset = LeafData(files=images,
                         transform=augs)

# data loader
image_loader = DataLoader(image_dataset,
                          batch_size=batch_size,
                          shuffle=False,
                          num_workers=num_workers,
                          pin_memory=True)

####### COMPUTE MEAN / STD

# placeholders
psum = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs in tqdm(image_loader):
    psum += inputs.sum(axis=[0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis=[0, 2, 3])

####### FINAL CALCULATIONS

# pixel count
count = len(images) * image_size * image_size

# mean and std
total_mean = psum / count
total_var = (psum_sq / count) - (total_mean ** 2)
total_std = torch.sqrt(total_var)

total_mean_255 = total_mean * 255
total_std_255 = total_std * 255

# output
print('mean: ' + str(total_mean))
print('std:  ' + str(total_std))
print('mean_255: ' + str(total_mean_255))
print('std_255:  ' + str(total_std_255))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 362/362 [02:10<00:00,  2.78it/s]


mean: tensor([0.7666, 0.7488, 0.7744])
std:  tensor([0.2579, 0.2621, 0.2480])
mean_255: tensor([195.4933, 190.9400, 197.4650])
std_255:  tensor([65.7644, 66.8242, 63.2313])
